In [2]:
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [3]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = self.linear3(x)
        return x



In [4]:
import copy
import numpy as np
seed_torch(seed)
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 50, 5, 10, 3, 3
model = Net(D_in, H1, H2, D_out)
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
init_state = copy.deepcopy(model.state_dict())
init_state_opt = copy.deepcopy(optimizer.state_dict())


In [13]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm

import xlrd
import pandas as pd
book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,9)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)
book = xlrd.open_workbook('prepareddatakfold.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_replicated = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7200)]

scaler = StandardScaler()
scaler.fit(data_unreplicated)
standardised_data = scaler.transform(data_replicated)

x_train = standardised_data[:, 0:5]
y_train = standardised_data[:, 5:8]

n_splits = 3
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))



In [18]:
i = 2
batches = 50
epoch_range = [2000]
for q in range (3,15,3):
    for l in range (5,25,5):
            N, D_in, H1, H2, D_out = batches, 5, l, q, 3
            model = Net(D_in, H1, H2, D_out)
            model.train()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            init_state = copy.deepcopy(model.state_dict())
            init_state_opt = copy.deepcopy(optimizer.state_dict())
            av_opt_epoch=0
            av_opt_loss=0
            for i, (train_idx, valid_idx) in enumerate(splits):
                model.load_state_dict(init_state)
                optimizer.load_state_dict(init_state_opt)
                x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
                y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
                x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
                y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
                train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
                valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batches,
                                                           shuffle=True)
                valid_loader = torch.utils.data.DataLoader(valid, batch_size=batches,
                                                           shuffle=False)
                Gp=0
                epoch=0
                opt_loss=1
                while Gp<1 and epoch<200:
                    epoch+=1
                    model.train()
                    for x_batch, y_batch in train_loader:
                        y_pred = model(x_batch)
                        loss = loss_fn(y_pred, y_batch)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    model.eval()
                    avg_val_loss = 0.
                    for x_batch, y_batch in valid_loader:
                        y_pred = model(x_batch).detach()
                        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader.dataset)
                    if avg_val_loss<opt_loss:
                        opt_loss = avg_val_loss  
                        opt_epoch=epoch
                    Gp=(avg_val_loss/opt_loss)-1
                av_opt_epoch+=opt_epoch/3
                av_opt_loss+=opt_loss/3
            print(H1,H2,av_opt_epoch,av_opt_loss)
         


5 3 95.33333333333333 0.6650887845704951
10 3 44.66666666666667 0.7058578373657332
15 3 98.0 0.6984000955190923
20 3 66.33333333333333 0.6797964488052659
5 6 52.66666666666667 0.6033962967081203
10 6 43.0 0.6553343290256131
15 6 41.0 0.6515493819283114
20 6 47.333333333333336 0.6203191373497249
5 9 60.333333333333336 0.6731260391987032
10 9 30.333333333333336 0.6189615372568369
15 9 37.666666666666664 0.6297398509664668
20 9 77.66666666666667 0.6258628155208298
5 12 57.66666666666667 0.6101373361630572
10 12 28.333333333333336 0.624052457511425
15 12 78.0 0.5849493671622541
20 12 25.333333333333336 0.6664278575529655


In [7]:
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANN3.pt"))                
                                